In [5]:
!pip install datasets --quiet
!pip install transformers --quiet
!pip install gensim==3.8.3 --quiet
!pip install tensorflow==2.8.2
!pip install -U tensorflow_text==2.8.2 --quiet
!pip install tensorflow-datasets --quiet
!pip install pydot --quiet

     |████████████████████████████████| 441 kB 35.3 MB/s 
     |████████████████████████████████| 95 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 67.1 MB/s 
     |████████████████████████████████| 115 kB 70.3 MB/s 
     |████████████████████████████████| 163 kB 74.3 MB/s 
     |████████████████████████████████| 127 kB 75.3 MB/s 
     |████████████████████████████████| 115 kB 76.6 MB/s 
     |████████████████████████████████| 5.5 MB 29.0 MB/s 
     |████████████████████████████████| 7.6 MB 54.2 MB/s 
     |████████████████████████████████| 24.2 MB 1.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 437.8 MB 38 kB/s 
     |████████████████████████████████| 5.8 MB 30.2 MB/s 
     |████████████████████████████████| 462 kB 74.4 MB/s 
     |████████████████████████████████| 1.4 MB 54.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-es

     |████████████████████████████████| 4.9 MB 22.8 MB/s 


In [6]:
import os
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text
import pandas as pd
import transformers
import gensim

from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from pprint import pprint 
from datasets import load_dataset
from nltk.corpus import reuters
from nltk.data import find
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter
from transformers import BertTokenizer, TFBertModel

ImportError: ignored

In [ ]:
url = 'https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv'
df = pd.read_csv(url, index_col=0)
df

# Field Description from  Davidson et.al github

count = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF).

hate_speech = number of CF users who judged the tweet to be hate speech.

offensive_language = number of CF users who judged the tweet to be offensive.

neither = number of CF users who judged the tweet to be neither offensive nor non-offensive.

class = class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither

In [ ]:
df.describe()

In [ ]:
df['class'].hist()

Very large skew towards offensive langauge in the dataset

In [ ]:
training_tweets, testing_tweets = train_test_split(df, test_size=0.2, random_state=25, stratify = df['class'])
valid_tweets, testing_tweets = train_test_split(testing_tweets, test_size = 0.5, random_state=25, stratify = testing_tweets['class'])

In [ ]:
print(f"No. of training examples: {training_tweets.shape[0]}")
print(f"No. of validation examples: {valid_tweets.shape[0]}")
print(f"No. of testing examples: {testing_tweets.shape[0]}")

In [ ]:
training_tweets['class'].hist()

In [ ]:
valid_tweets['class'].hist()

In [ ]:
testing_tweets['class'].hist()

In [ ]:
# get average length of tweets in the various sets
print(training_tweets['tweet'].apply(lambda x: len(x.split(' '))).mean())
print(valid_tweets['tweet'].apply(lambda x: len(x.split(' '))).mean())
print(testing_tweets['tweet'].apply(lambda x: len(x.split(' '))).mean())


In [ ]:
max_length = 15

CNN Baseline

In [ ]:
nltk.download('word2vec_sample')
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [ ]:
EMBEDDING_DIM = len(model['university']) 

# initialize embedding matrix and word-to-id map:
embedding_matrix = np.zeros((len(model.vocab.keys()) + 1, EMBEDDING_DIM))       
vocab_dict = {}

# build the embedding matrix and the word-to-id map:
for i, word in enumerate(model.vocab.keys()):
    embedding_vector = model[word]
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        vocab_dict[word] = i

vocab_dict['[UNK]'] = len(vocab_dict)

In [ ]:
embedding_matrix.shape

In [ ]:
tokenizer = tf_text.WhitespaceTokenizer()
train_tokens = tokenizer.tokenize(training_tweets['tweet'])
valid_tokens = tokenizer.tokenize(valid_tweets['tweet'])
test_tokens = tokenizer.tokenize(testing_tweets['tweet'])

In [ ]:
train_tokens[0]

In [ ]:
MAX_SEQUENCE_LENGTH=15

In [ ]:
def docs_to_vocab_ids(tokenized_texts_list):
    """
    converting a list of strings to a list of lists of word ids
    """
    texts_vocab_ids = []
    for i, token_list in enumerate(tokenized_texts_list):

        # Get the vocab id for each token in this doc ([UNK] if not in vocab)
        vocab_ids = []
        for token in list(token_list.numpy()):
            decoded = token.decode('utf-8', errors='ignore')
            if decoded in vocab_dict:
                vocab_ids.append(vocab_dict[decoded])
            else:
                vocab_ids.append(vocab_dict['[UNK]'])
            
        # Truncate text to max length, add padding up to max length
        vocab_ids = vocab_ids[:MAX_SEQUENCE_LENGTH]
        n_padding = (MAX_SEQUENCE_LENGTH - len(vocab_ids))
        # For simplicity in this model, we'll just pad with uknown tokens
        vocab_ids += [vocab_dict['[UNK]']] * n_padding

        # Add this example to the list of converted docs
        texts_vocab_ids.append(vocab_ids)
            
        if i % 5000 == 0:
            print('Examples processed: ', i)

    return np.array(texts_vocab_ids)

In [ ]:
train_input = docs_to_vocab_ids(train_tokens)
valid_input = docs_to_vocab_ids(valid_tokens)
test_input = docs_to_vocab_ids(test_tokens)

train_labels = np.array(training_tweets['class'])
valid_labels = np.array(valid_tweets['class'])
test_labels = np.array(testing_tweets['class'])

In [ ]:
train_labels[1]

In [ ]:
train_input[1]

In [ ]:
def build_cnn_model(num_filters, kernel_sizes, dense_layer_dims, dropout_rate, embeddings_trainable=False):
    cnn_input_layer = tf.keras.layers.Input(shape=(max_length,), dtype='int64')
    cnn_embedding_layer = Embedding(embedding_matrix.shape[0],
                                    embedding_matrix.shape[1],
                                    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                    input_length=max_length,
                                    trainable=embeddings_trainable)
    cnn_embeddings = cnn_embedding_layer(cnn_input_layer)

    conv_layers_for_all_kernel_sizes = []
    for kernel_size, filters in zip(kernel_sizes, num_filters):
        conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(cnn_embeddings)
        conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
        conv_layers_for_all_kernel_sizes.append(conv_layer)
    
    conv_output = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)
    last_hidden_output = keras.layers.Dropout(rate=dropout_rate)(conv_output)

    for dense_layer_dim in dense_layer_dims:
        last_hidden_output = keras.layers.Dense(dense_layer_dim, activation='relu')(last_hidden_output)
    
    cnn_prediction = keras.layers.Dense(3, activation='softmax')(last_hidden_output)

    cnn_model = keras.Model(inputs=cnn_input_layer, outputs=cnn_prediction)
    cnn_model.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
                      metrics=['accuracy'])
    return cnn_model

In [ ]:
# Now specify model hyperparameters and call the function to create a CNN model

num_filters = [100]
kernel_sizes = [ 10]
dense_layer_dims = [50]
dropout_rate = 0.5

cnn_model = build_cnn_model(num_filters, kernel_sizes, dense_layer_dims, dropout_rate)

In [ ]:
cnn_model.summary()

In [ ]:
cnn_history = cnn_model.fit(train_input,
                            train_labels,
                            validation_data=(valid_input, valid_labels),
                            batch_size=32,
                            epochs=5
                            )

In [ ]:
score = cnn_model.evaluate(test_input, test_labels)
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

BERT

In [ ]:
#make it easier to use a variety of BERT subword models
model_checkpoint = 'bert-base-cased'

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
bert_model = TFBertModel.from_pretrained(model_checkpoint)

In [ ]:
# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length` and return a tf Tensor
train_encodings = bert_tokenizer(list(training_tweets['tweet']), truncation=True, padding=True, max_length=max_length, return_tensors='tf')
valid_encodings = bert_tokenizer(list(valid_tweets['tweet']), truncation=True, padding=True, max_length=max_length, return_tensors='tf')
test_encodings = bert_tokenizer(list(testing_tweets['tweet']), truncation=True, padding=True, max_length=max_length, return_tensors='tf')

In [ ]:
bert_model.summary()

In [ ]:
def create_bert_multiclass_model(checkpoint = model_checkpoint,
                                hidden_size = 91, 
                                dropout=0.3,
                                learning_rate=0.00005):
  """
  Build a simple classification model with BERT. Use the Pooled Output for classification purposes.
  """
  bert_model = TFBertModel.from_pretrained(checkpoint)                                              
  ### YOUR CODE HERE

  input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer')
  token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
  attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

  bert_inputs = {'input_ids': input_ids,
  'token_type_ids': token_type_ids,
  'attention_mask': attention_mask
  }      

  bert_out = bert_model(bert_inputs)

  pooled_token = bert_out[1]

  hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooled_token)


  hidden = tf.keras.layers.Dropout(dropout)(hidden)  


  classification = tf.keras.layers.Dense(3, activation='softmax',name='classification_layer')(hidden)

  classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

  classification_model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
      metrics=['accuracy'])


  ### END YOUR CODE
  return classification_model

In [ ]:
pooled_bert_model = create_bert_multiclass_model(checkpoint = model_checkpoint)

In [ ]:
pooled_bert_model.summary()

In [ ]:
#It takes 6 minutes to complete an epoch when using a GPU
pooled_bert_model_history = pooled_bert_model.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask], 
                                                  np.array(training_tweets['class']),   
                                                  validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask], 
                                                  np.array(valid_tweets['class'])),    
                                                  batch_size=8, 
                                                  epochs=5)  

In [ ]:
score = pooled_bert_model.evaluate([test_encodings.input_ids, test_encodings.token_type_ids, test_encodings.attention_mask], 
                                                  np.array(testing_tweets['class'])) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

In [ ]:
#run and capture all predictions from our test set using model.predict
### YOUR CODE HERE
predictions = pooled_bert_model.predict([test_encodings.input_ids, test_encodings.token_type_ids, test_encodings.attention_mask])
### END YOUR CODE

#now we need to get the highest probability in the distribution for each prediction
#and store that in a tf.Tensor
predictions = tf.argmax(predictions, axis=-1)
predictions

In [ ]:
print(classification_report(testing_tweets['class'], predictions.numpy(), target_names=['0','1','2']))